# Intro to Julia

This notebook is an introduction to the Julia language and its commonly used IJulia/Jupyter notebook interface. 

Source: [Irra Na and Lea Kapelevich](https://github.com/adelarue/cos_2021/blob/main/6%2B7_julia_and_jump/Intro%20to%20Julia-%20complete.ipynb)

It is based on:
- material developed by Miles Lubin and Sebastien Martin of the MIT Operations Research Center
- Arthur Delarue's COS 2018 notebook (https://github.com/PhilChodrow/cos_2018/tree/master/6_julia_and_jump)
- online open-source material (http://ucidatascienceinitiative.github.io/IntroToJulia/Html/WhyJulia)
- Christopher Rackauckas' fantastic blogs (https://www.stochasticlifestyle.com/7-julia-gotchas-handle/)

# First of all... what is this thing? It's an IJulia Notebook!

Repurposes Python's Jupyter notebook!

_Click `Help -> User Interface Tour`_

**Think of the notebook as a document that can interact with your computer.** The *document* relies only on a modern browser for rendering. When you connect the document to a Julia kernel and terminal instance on a computer, however, the document can send any command to the computer and show any output (text or graphics). 

* Each notebook is composed of cells
* Two modes:
    * Command Mode for creating or deleting cells, saving or renaming the notebook, and other application-level functions
    * Edit Mode for manipulating text in individual cells
* Create a cell by:
    * Clicking `Insert >> Insert Cell`
    * Pressing `a` or `b` in Command Mode (`a` = "above", `b` = "below")
    * Pressing `Alt/Option+Enter` in Edit Mode
* Delete a cell by:
    * Clicking `Edit >> Delete Cell`
    * Pressing `dd`
* Execute a cell by:
    * Clicking `Cell >> Run`
    * Pressing `Ctrl+Enter`
    * Pressing `Shift+Enter`

Other functions:
* Undo last text edit with `Ctrl+z` in Edit Mode
* Undo last cell manipulation with `z` in Command Mode
* Save notebook with `Ctrl+s` in Edit Mode
* Save notebook with `s` in Command Mode

Though notebooks rely on your browser to work, they do not require an internet connection. The only online tool that is consistently used is MathJax (for math rendering).


### Get comfortable with the notebook
Notebooks are designed to not be fragile. If you try to close a notebook with unsaved changes, the browser will warn you.

Try the following exercises on your own time:

>**\[Exercise\]**: Close/open
>1. Save the notebook
>2. Copy the address
>3. Close the tab
>4. Paste the address into a new tab (or re-open the last closed tab with `Ctrl+Shift+T` on Chrome)
>_The document is still there, and the Julia kernel is still alive! Nothing is lost._

>**\[Exercise\]**: Zoom
>Try changing the magnification of the web page (`Ctrl+, Ctrl-` on Chrome).
>_Text and math scale well (so do graphics if you use an SVG or PDF backend)._

>**\[Exercise\]**: MathJax
>1. Create a new cell, and select the type `Markdown` (or press `m`)
>2. Type an opening \\$, your favorite mathematical expression, and a closing \\$.
>3. Run the cell to render the $\LaTeX$ expression.
>4. Right-click the rendered expression.

## Why Julia?

A **high-level** language:

Easy to use and learn, with a similar syntax to Python/Matlab.
It is possible to do complicated computations quickly.
For example, Solving $Ax = b$ with $A = \begin{pmatrix}
 1 & 2 &  3\\ 
 2 & 1 & 2\\ 
 3 & 2 & 1
\end{pmatrix}$ and $b = \begin{pmatrix}
 1 \\ 
 1 \\ 
 1 
\end{pmatrix}$ is as simple as:

In [1]:
A = [1 2 3
     2 1 2
     3 2 1]

b = [1,1,1]
A \ b

3-element Vector{Float64}:
 0.25
 0.0
 0.25000000000000006

A **dynamic** language:

Julia is, like Python, Matlab or R, a dynamic language: you can interact with the language without the need to compile your code. Static or compiled languages, like C or Fortran, are more complicated to use but generally faster, and thus used when there is a need for time-efficient computations.

Two-language approach: use high level languages for research and scripting, then translate the final result into a static language for performance.

A **high-performance** language:

Julia is fast. Thanks to multiple dispatch, a strong type system, and just-in-time compilation, it can reach performance comparable to C and Fortran.


![figures/Julia-benchmarks.png](figures/Julia-benchmarks.png)

A language for **technical computing**:

- Julia has a lot of built in functions for scientific computing.
- A growing number of packages, mostly written in Julia itself.
- More and more users in Finance, Biology, Optimization.
- Can run C and Python code seamlessly (using Scikit for Machine Learning...)

In this notebook we will try to cover some gotchas and good workflow practices to get the most out of Julia. 

## Warm-up exercise

Complete `find_entering_var` below, which returns the minimum reduced cost and index of the entering variable (with the minimum reduced cost), inside an iteration of the simplex method. 

If no variable has negative reduced cost, we will simply return zeros for `min_rc` and `min_idx`. If multiple variables have the lowest reduced cost, we will return the last of these.

Remember the vector of reduced costs is given by:
$$
rc = c_N - A'\pi
$$
and the $i^{th}$ reduced cost is
$$
rc_i = c_i - A_i' \pi
$$
where $A_i$ is the $i^{th}$ column of $A$.

You might find it useful to use the function `dot` from the `LinearAlgebra` library.

In [1]:
using LinearAlgebra
dot([1, 2], [3, 4])

11

In [3]:
function find_entering_var(A::Matrix{Float64}, c::Vector{Float64}, pi::Vector{Float64}, var_status::Vector{Int})
    min_rc = 0
    min_idx = 0
    for k in eachindex(var_status)
        # only check nonbasic variables
        if iszero(var_status[k])
            rc = c[k] - dot(A[:, k], pi)
            if rc < min_rc
                min_rc = rc
                min_idx = k
            end
        end
    end
    return (min_rc, min_idx)
end

find_entering_var (generic function with 1 method)

In [4]:
# test your function by running this cell

using Random
function make_data(T::Type)
Random.seed!(1)
    basic_idxs = [2, 4, 6]
    A = T[3 2 1 2 1 0 0; 1 1 1 1 0 1 0; 4 3 3 4 0 0 1]
    B = A[:, basic_idxs]
    B_inv = inv(B) # note this would never happen inside the algorithm, we always have B_inv available
    b = T[225, 117, 420]
    c = -T[19, 13, 12, 17, 0, 0, 0]
    c_b = c[basic_idxs]
    x_b = B_inv * b
    var_status = [0, 1, 0, 2, 0, 3]
    pi = B_inv' * c_b
    return (A, b, c, B_inv, pi, var_status, basic_idxs)
end
(A, b, c, B_inv, pi, var_status, basic_idxs) = make_data(Float64)

find_entering_var(A, c, pi, var_status) # should be (-1.5, 1)

(-1.5, 1)

## A bit more on why Julia is fast

In [5]:
# When we write a function, it can have many "methods"
+(1, 2)

3

In [6]:
methods(+)

# 208 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:529
[3] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:535
[4] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:390
[5] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Dates/src/periods.jl:372
[8] +(x::T, y::Integer) where T<:AbstractChar in Base at char.jl:237
[9] +(A::Bidiagonal, B::Bidiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/bidiag.jl:354
[10] +(x::Bidiagonal, H::UpperHessenberg) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:107
[11] +(A::Bidiagonal, B::UpperTriangular) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:94
[12] +(A::Bidiagonal, B::UnitUpperTriangular) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:94
[13] +(A::Bidiagonal, B::LowerTriangular) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:94
[14] +(A::Bidiagonal, B::UnitLowerTriangular) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:94
[15] +(A::Bidiagonal, B::Diagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:115
[16] +(A::Bidiagonal, B::Tridiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:183
[17] +(A::Bidiagonal, B::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:203
[18] +(A::Bidiagonal{var"#s859", V} where {var"#s859"<:Number, V<:AbstractVector{var"#s859"}}, B::UniformScaling) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:236
[19] +(x::Base.TwicePrecision, y::Number) in Base at twiceprecision.jl:279
[20] +(x::Base.TwicePrecision{T}, y::Base.TwicePrecision{T}) where T in Base at twiceprecision.jl:285
[21] +(x::Base.TwicePrecision, y::Base.TwicePrecision) in Base at twiceprecision.jl:290
[22] +(Da::Diagonal, Db::Diagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:172
[23] +(D::Diagonal, S::Symmetric) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:176
[24] +(D::Diagonal{var"#s859", V} where {var"#s859"<:Real, V<:AbstractVector{var"#s859"}}, H::Hermitian) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:182
[25] +(x::Diagonal, H::UpperHessenberg) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:107
[26] +(A::Diagonal, B::Bidiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:125
[27] +(A::Diagonal, B::SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:135
[28] +(A::Diagonal, B::Tridiagonal) in LinearAlgebra at /

In [7]:
function my_function(x)
    println("Default output")
end

function my_function(x::Int) # only called when x is an integer
    println("You gave me an integer!")
end

methods(my_function)

# 2 methods for generic function "my_function":
[1] my_function(x::Int64) in Main at In[7]:5
[2] my_function(x) in Main at In[7]:1

In [8]:
my_function(1.0)
my_function(1)
my_function("ORC")

Default output
You gave me an integer!
Default output


In [9]:
# you can check which method will be dispatched to with @which
@which +(1, 2)

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87

We could have made `find_entering_var` more general e.g. we could have a simplex running in rational numbers, arbitrary precision, etc.!

In [10]:
# E.g. 
1 // 2 # fraction in Julia
typeof(1 // 2)

Rational{Int64}

In [6]:
a = BigFloat(1.24)

1.2399999999999999911182158029987476766109466552734375

In [11]:
function find_entering_var(A::Matrix{T}, c::Vector{T}, pi::Vector{T}, var_status::Vector{Int}) where {T <: Real}
    min_rc = 0
    min_idx = 0
    for k in eachindex(var_status)
        # only check nonbasic variables
        if iszero(var_status[k])         
            rc = c[k] - dot(A[:, k], pi)
            if rc < min_rc
                min_rc = rc
                min_idx = k
            end
        end
    end
    return (min_rc, min_idx)
end

find_entering_var (generic function with 2 methods)

In [12]:
# let's generate some rational data
(A, b, c, B_inv, pi, var_status, basic_idxs) = make_data(Rational{Int})
@show A
@show b
@show c
;

A = Rational{Int64}[3//1 2//1 1//1 2//1 1//1 0//1 0//1; 1//1 1//1 1//1 1//1 0//1 1//1 0//1; 4//1 3//1 3//1 4//1 0//1 0//1 1//1]
b = Rational{Int64}[225//1, 117//1, 420//1]
c = Rational{Int64}[-19//1, -13//1, -12//1, -17//1, 0//1, 0//1, 0//1]


In [13]:
# test that this "just works" by running this cell
(min_rc, min_idx) = find_entering_var(A, c, pi, var_status) # should be (-3//2, 1)

(-3//2, 1)

In [14]:
function count_up(n)
    count = 0.0
    for i in 1:n
        count += sin(1.0) + cos(1.0) + tan(1.0)
    end
    return count
end
println("First use: slower")
@time count_up(1e7) 
println("Second use: compiled and optimized automatically")
@time count_up(1e7);
println("Third use: compiled and optimized automatically")
@time count_up(1e7);

First use: slower
  0.011430 seconds
Second use: compiled and optimized automatically
  0.010660 seconds
Third use: compiled and optimized automatically
  0.010364 seconds


Aside: always run twice when using @time

Aside \#2: this demonstration used to be more dramatic in earlier versions of Julia. Now this function is just kind of fast no matter what.

### What is type stability?

A function is type stable if the type of the output depends only on the types of the inputs, not their values.

In [20]:
@code_warntype find_entering_var(A, c, pi, var_status)

MethodInstance for find_entering_var(::Matrix{Rational{Int64}}, ::Vector{Rational{Int64}}, ::Vector{Rational{Int64}}, ::Vector{Int64})
  from find_entering_var(A::Matrix{T}, c::Vector{T}, pi::Vector{T}, var_status::Vector{Int64}) where T<:Real in Main at In[14]:1
Static Parameters
  T = Rational{Int64}
Arguments
  #self#::Core.Const(find_entering_var)
  A::Matrix{Rational{Int64}}
  c::Vector{Rational{Int64}}
  pi::Vector{Rational{Int64}}
  var_status::Vector{Int64}
Locals
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  min_idx::Int64
  min_rc::Union{Rational{Int64}, Int64}
  k::Int64
  rc::Rational{Int64}
Body::Tuple{Union{Rational{Int64}, Int64}, Int64}
1 ─       (min_rc = 0)
│         (min_idx = 0)
│   %3  = Main.eachindex(var_status)::Base.OneTo{Int64}
│         (@_6 = Base.iterate(%3))
│   %5  = (@_6 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #7 if not %6
2 ┄       Core.NewvarNode(:(rc))
│   %9  = @_6::Tuple{Int64, Int64}
│         (k = Core.getfield(%9, 1))
│ 

We see that `A`, `c`, and `pi` are rational, but the type of `min_rc` is not necessarily fixed throughout our function.

Let's try to fix our function to get rid of the red.

In [15]:
function find_entering_var(A::Matrix{T}, c::Vector{T}, pi::Vector{T}, var_status::Vector{Int}) where {T <: Real}
    min_rc = zero(T) # <----------------
    min_idx = 0
    for k in eachindex(var_status)
        # only check nonbasic variables
        if iszero(var_status[k])
            rc = c[k] - dot(A[:, k], pi)
            if rc < min_rc
                min_rc = rc
                min_idx = k
            end
        end
    end
    return (min_rc, min_idx)
end

@code_warntype find_entering_var(A, c, pi, var_status)

MethodInstance for find_entering_var(::Matrix{Rational{Int64}}, ::Vector{Rational{Int64}}, ::Vector{Rational{Int64}}, ::Vector{Int64})
  from find_entering_var(A::Matrix{T}, c::Vector{T}, pi::Vector{T}, var_status::Vector{Int64}) where T<:Real in Main at In[15]:1
Static Parameters
  T = Rational{Int64}
Arguments
  #self#::Core.Const(find_entering_var)
  A::Matrix{Rational{Int64}}
  c::Vector{Rational{Int64}}
  pi::Vector{Rational{Int64}}
  var_status::Vector{Int64}
Locals
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  min_idx::Int64
  min_rc::Rational{Int64}
  k::Int64
  rc::Rational{Int64}
Body::Tuple{Rational{Int64}, Int64}
1 ─       (min_rc = Main.zero($(Expr(:static_parameter, 1))))
│         (min_idx = 0)
│   %3  = Main.eachindex(var_status)::Base.OneTo{Int64}
│         (@_6 = Base.iterate(%3))
│   %5  = (@_6 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #7 if not %6
2 ┄       Core.NewvarNode(:(rc))
│   %9  = @_6::Tuple{Int64, Int64}
│         (k = Core.getfield

Other useful functions include `eltype()` or `one()`.

For those interested, read more [here](https://arxiv.org/abs/2109.01950).

## Exercise
Complete the function `find_leaving_var` to return `(min_ratio, min_idx)`. I.e. the minimum and the minimizer of:
$$
\min_{k: e_k' B^{-1} A_i > 0} \frac{e_k' B^{-1}b}{e_k' B^{-1} A_i}
$$
If $ e_k' B^{-1} A_i \leq 0 $ for all $k$, return (0, Inf). Assume you are provided the vectors `B_inv_A_i = B \ A_i`, and `x_b = B \ b`, as well as a list of basic indices as input.

Test for correctness and type stability by running the box below. 

In [22]:
# hint:
@show typeof(Inf)
@show typeof(Float64(Inf))
@show typeof(Rational{Int}(Inf))

typeof(Inf) = Float64
typeof(Float64(Inf)) = Float64
typeof(Rational{Int}(Inf)) = Rational{Int64}


Rational{Int64}

In [16]:
function find_leaving_var(x_b::Vector{T}, B_inv_A_i::Vector{T}, basic_idxs::Vector{Int}) where {T <: Real}
    min_ratio = T(Inf)
    min_idx = 0
    for k in eachindex(B_inv_A_i)
        if B_inv_A_i[k] > 0
            ratio = x_b[k] / B_inv_A_i[k]
            if ratio < min_ratio
                min_ratio = ratio
                min_idx = k
            end
        end
    end
    return (min_ratio, min_idx)
end

find_leaving_var (generic function with 1 method)

In [17]:
# use our data and entering variable from before
(A, b, c, B_inv, pi, var_status, basic_idxs) = make_data(Float64)
(_, entering_idx) = find_entering_var(A, c, pi, var_status)
x_b = B_inv * b
B_inv_A_i = B_inv * A[:, entering_idx]


(min_ratio, leaving_idx) = find_leaving_var(x_b, B_inv_A_i, basic_idxs) # should be (14.999999999999993, 1)

(14.999999999999993, 1)

In [18]:
@code_warntype find_leaving_var(x_b, B_inv_A_i, basic_idxs)

MethodInstance for find_leaving_var(::Vector{Float64}, ::Vector{Float64}, ::Vector{Int64})
  from find_leaving_var(x_b::Vector{T}, B_inv_A_i::Vector{T}, basic_idxs::Vector{Int64}) where T<:Real in Main at In[16]:1
Static Parameters
  T = Float64
Arguments
  #self#::Core.Const(find_leaving_var)
  x_b::Vector{Float64}
  B_inv_A_i::Vector{Float64}
  basic_idxs::Vector{Int64}
Locals
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  min_idx::Int64
  min_ratio::Float64
  k::Int64
  ratio::Float64
Body::Tuple{Float64, Int64}
1 ─       (min_ratio = ($(Expr(:static_parameter, 1)))(Main.Inf))
│         (min_idx = 0)
│   %3  = Main.eachindex(B_inv_A_i)::Base.OneTo{Int64}
│         (@_5 = Base.iterate(%3))
│   %5  = (@_5 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #7 if not %6
2 ┄       Core.NewvarNode(:(ratio))
│   %9  = @_5::Tuple{Int64, Int64}
│         (k = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = Base.getindex(B_inv_A_i, k)::Float64
│   %13 = (%1

### Using the REPL

Working in the REPL doesn't allow type specification:

In [40]:
a = 3
f() = a
@code_warntype f()

MethodInstance for f()
  from f() in Main at In[40]:2
Arguments
  #self#::Core.Const(f)
Body::Any
1 ─     return Main.a



In [41]:
# wrap code inside functions/modules when possible
function foo()
    a = 3
    f() = a
    return f()
end
@code_warntype foo()

MethodInstance for foo()
  from foo() in Main at In[41]:2
Arguments
  #self#::Core.Const(foo)
Locals
  f::var"#f#2"{Int64}
  a::Int64
Body::Int64
1 ─      (a = 3)
│   %2 = Main.:(var"#f#2")::Core.Const(var"#f#2")
│   %3 = Core.typeof(a::Core.Const(3))::Core.Const(Int64)
│   %4 = Core.apply_type(%2, %3)::Core.Const(var"#f#2"{Int64})
│        (f = %new(%4, a::Core.Const(3)))
│   %6 = (f::Core.Const(var"#f#2"{Int64}(3)))()::Core.Const(3)
└──      return %6



In [42]:
const my_const = 3
f() = my_const
@code_warntype f()

MethodInstance for f()
  from f() in Main at In[42]:2
Arguments
  #self#::Core.Const(f)
Body::Int64
1 ─     return Main.my_const



## Matrices and views

In [20]:
A = [3 5; 8 12]
@show A
B = A
B[1, 1] = 1
@show A
@show B;

A = [3 5; 8 12]
A = [1 5; 8 12]
B = [1 5; 8 12]


In [22]:
# using Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools
A = randn(100, 100); # random 100x100 matrix with elements from N(0,1)
@benchmark B = A # set the value of B to point to the same location as A

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.228 ns … 50.551 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.332 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.463 ns ±  1.298 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █▅▅▄▃▆▂▅▂▄▄▆▁                                             ▂
  ▄█████████████▇▇▇███▇█▇█▆█▇█▇▆▆▅▅▆▆▆▆▅▅▆▆▅▆▇▆▆▅▃▅▆▅▆▆▆▅▆▆▅ █
  1.23 ns      Histogram: log(frequency) by time     2.42 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [23]:
B = A[1:2, 44:67]
@show B[1, 1]
@show A[1, 44]
B[1, 1] = 55
@show B[1, 1]
@show A[1, 44]

B[1, 1] = -0.00973495828347777
A[1, 44] = -0.00973495828347777
B[1, 1] = 55.0
A[1, 44] = -0.00973495828347777


-0.00973495828347777

In [24]:
@benchmark B = A[1:2, 44:67]

BenchmarkTools.Trial: 10000 samples with 875 evaluations.
 Range (min … max):  131.250 ns …   3.655 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     147.378 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   183.361 ns ± 140.220 ns  ┊ GC (mean ± σ):  5.62% ± 7.95%

  ▁█▃                                                            
  ███▅▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▂▂ ▃
  131 ns           Histogram: frequency by time          531 ns <

 Memory estimate: 448 bytes, allocs estimate: 1.

If we want to operate on `A[1:2, 44:67]`, using `A[1:2, 44:67]` creates a copy of the indexed elements in `A`. To avoid expensive copying, we can use the convenient `@views` macro.

In [25]:
@views B = A[1:2, 44:67]
@show B[1, 1]
@show A[1, 44]
B[1, 1] = 55
@show B[1, 1]
@show A[1, 44]

B[1, 1] = -0.00973495828347777
A[1, 44] = -0.00973495828347777
B[1, 1] = 55.0
A[1, 44] = 55.0


55.0

### Operations on arrays

In [53]:
f(x) = 2x

f (generic function with 2 methods)

In [54]:
using Random
Random.seed!(1)
n = 500
my_vec = randn(n);

In [55]:
@benchmark vec2 = f(my_vec)

BenchmarkTools.Trial: 10000 samples with 237 evaluations.
 Range (min … max):  279.599 ns … 37.720 μs  ┊ GC (min … max):  0.00% … 96.65%
 Time  (median):     664.321 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   979.112 ns ±  2.516 μs  ┊ GC (mean ± σ):  37.69% ± 14.07%

  ▆█▂                                                          ▁
  ███▆▆▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▁▃▄▄▅▅▅▆▇▇▇ █
  280 ns        Histogram: log(frequency) by time      17.2 μs <

 Memory estimate: 4.06 KiB, allocs estimate: 1.

In [56]:
@benchmark vec2 = f.(my_vec)

BenchmarkTools.Trial: 10000 samples with 172 evaluations.
 Range (min … max):  609.140 ns … 35.512 μs  ┊ GC (min … max):  0.00% … 97.37%
 Time  (median):     943.991 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.299 μs ±  3.003 μs  ┊ GC (mean ± σ):  28.44% ± 11.79%

  █▆▃                                                          ▁
  ███▇▄▃▁▁▁▁▁▁▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▅▄▅▅▅▃▅▆ █
  609 ns        Histogram: log(frequency) by time      23.7 μs <

 Memory estimate: 4.12 KiB, allocs estimate: 4.

In [59]:
(A, b, c, B_inv, pi, var_status, basic_idxs) = make_data(Float64)
@benchmark find_entering_var(A, c, pi, var_status)

BenchmarkTools.Trial: 10000 samples with 611 evaluations.
 Range (min … max):  183.646 ns …   7.371 μs  ┊ GC (min … max): 0.00% … 96.87%
 Time  (median):     209.065 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   240.330 ns ± 359.813 ns  ┊ GC (mean ± σ):  9.15% ±  5.91%

      ▄▆█▇▆▄▃▂▁▁▁▁                                              ▂
  ▆▇█▇██████████████▇███▆███▇▇▇▇▇▆▆▆▆▅▆▆▇▇▆▆▆▅▅▆▆▅▅▆▄▅▅▄▄▄▅▅▅▄▅ █
  184 ns        Histogram: log(frequency) by time        403 ns <

 Memory estimate: 288 bytes, allocs estimate: 5.

In [60]:
function find_entering_var2(A::Matrix{Float64}, c::Vector{Float64}, pi::Vector{Float64}, var_status::Vector{Int})
    min_rc = 0.0
    min_idx = 0
    for k in eachindex(var_status)
        # only check nonbasic variables
        if iszero(var_status[k])
            @views rc = c[k] - dot(A[:, k], pi)
            if rc < min_rc
                min_rc = rc
                min_idx = k
            end
        end
    end
    return (min_rc, min_idx)
end
@benchmark find_entering_var2($A, $c, $pi, $var_status)

BenchmarkTools.Trial: 10000 samples with 985 evaluations.
 Range (min … max):  48.577 ns … 415.958 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     52.247 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   54.330 ns ±  13.124 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂▁▇▇▃▁                                                      ▁
  ████████▆▄▆▆▆▇█▆█▇▇▆▆▅▅▅▅▅▆▆▆▅▆▆▆▆▇▇▇▇▅▆▅▆▅▅▆▅▆▆▆▆▆▅▆▅▅▅▄▄▃▄ █
  48.6 ns       Histogram: log(frequency) by time       115 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.